In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_financial_ratio")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              5759 non-null   int64              
 1   ticker          5759 non-null   object             
 2   stac_yymm       5759 non-null   object             
 3   grs             5759 non-null   object             
 4   bsop_prfi_inrt  5759 non-null   object             
 5   ntin_inrt       5759 non-null   object             
 6   roe_val         5759 non-null   object             
 7   eps             5759 non-null   object             
 8   sps             5759 non-null   object             
 9   bps             5759 non-null   object             
 10  rsrv_rate       5759 non-null   object             
 11  lblt_rate       5759 non-null   object             
 12  updated_at      5759 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(

,id,ticker,stac_yymm,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps,sps,bps,rsrv_rate,lblt_rate,updated_at
0,1,005930,202506,5.2900,-33.3600,-19.6200,6.64,1920.00,45533,58114.00,43513.3600,26.3600,2025-09-16 13:01:15.709244+00:00
1,2,005930,202503,10.0500,1.2000,21.7400,8.16,1186.00,46778,59027.00,44197.8800,26.9900,2025-09-16 13:01:15.709244+00:00
2,3,005930,202412,16.2000,398.3400,122.4500,9.03,4950.00,44293,57930.00,43743.2600,27.9300,2025-09-16 13:01:15.709244+00:00
3,4,005930,202409,17.7500,601.0000,192.0100,9.52,3834.00,44181,55376.00,41810.4500,27.1900,2025-09-16 13:01:15.709244+00:00
4,5,005930,202406,17.9700,1202.7900,403.1900,8.95,2394.00,42983,55011.00,41533.8400,26.6600,2025-09-16 13:01:15.709244+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,14.1600,80.4100,41.3100,7.95,95.00,17994,4928.00,418.3600,179.0600,2025-09-16 13:01:15.709244+00:00
5755,5756,002710,201812,5.5700,110.7000,548.7400,9.19,416.00,17582,4937.00,419.2400,171.5300,2025-09-16 13:01:15.709244+00:00
5756,5757,002710,201809,0.9200,72.6500,209.6400,7.76,257.00,17174,4768.00,401.5400,174.7500,2025-09-16 13:01:15.709244+00:00
5757,5758,002710,201806,-1.3200,56.7500,61.0500,5.68,124.00,16827,4660.00,390.1600,178.5000,2025-09-16 13:01:15.709244+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
for col in final_cols:
  df[col] .replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df[col].replace(['0', 0], np.nan, inplace=True)

missing_value_counts = df[final_cols].isnull().sum()

missing_value_counts

ticker            0
stac_yymm         0
grs               0
bsop_prfi_inrt    0
ntin_inrt         0
roe_val           0
eps               0
sps               6
bps               0
rsrv_rate         0
lblt_rate         0
updated_at        0
dtype: int64

In [5]:
missing_cols = missing_value_counts[missing_value_counts >= 100].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 12] - [결측 컬럼 : 0] = [후보 컬럼 : 12]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['updated_at']
[원본 컬럼 : 12] - [결측 컬럼 : 0] - [유니크 컬럼 : 1] = [후보 컬럼 : 11]


In [7]:
df[final_cols]

,ticker,stac_yymm,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps,sps,bps,rsrv_rate,lblt_rate
0,005930,202506,5.2900,-33.3600,-19.6200,6.64,1920.00,45533,58114.00,43513.3600,26.3600
1,005930,202503,10.0500,1.2000,21.7400,8.16,1186.00,46778,59027.00,44197.8800,26.9900
2,005930,202412,16.2000,398.3400,122.4500,9.03,4950.00,44293,57930.00,43743.2600,27.9300
3,005930,202409,17.7500,601.0000,192.0100,9.52,3834.00,44181,55376.00,41810.4500,27.1900
4,005930,202406,17.9700,1202.7900,403.1900,8.95,2394.00,42983,55011.00,41533.8400,26.6600
...,...,...,...,...,...,...,...,...,...,...,...
5754,002710,201903,14.1600,80.4100,41.3100,7.95,95.00,17994,4928.00,418.3600,179.0600
5755,002710,201812,5.5700,110.7000,548.7400,9.19,416.00,17582,4937.00,419.2400,171.5300
5756,002710,201809,0.9200,72.6500,209.6400,7.76,257.00,17174,4768.00,401.5400,174.7500
5757,002710,201806,-1.3200,56.7500,61.0500,5.68,124.00,16827,4660.00,390.1600,178.5000


In [8]:
print(final_cols)

['ticker', 'stac_yymm', 'grs', 'bsop_prfi_inrt', 'ntin_inrt', 'roe_val', 'eps', 'sps', 'bps', 'rsrv_rate', 'lblt_rate']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 재무 지표인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `stac_yymm` | 결산년월 | **필수** | 데이터가 어느 시점의 실적인지를 나타내는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 이벤트와 결합할 때 필수적입니다. |
| `grs` | 매출액증가율 | **상** | 기업의 외형적 성장세를 보여주는 가장 기본적인 지표입니다. 시장 기대치를 상회하거나 하회할 경우 주가 변동성에 직접적인 영향을 줍니다. |
| `bsop_prfi_inrt`| 영업이익증가율 | **상** | 기업의 핵심 영업활동으로 인한 이익의 성장성을 나타냅니다. 매출 성장보다 수익성의 질을 보여주므로 변동성 예측에 더 민감한 피처가 될 수 있습니다. |
| `ntin_inrt` | 순이익증가율 | **상** | 영업 외 손익까지 반영된 최종 이익의 성장률로, 기업의 총체적인 이익 창출 능력 변화를 나타내어 주가에 직접적인 영향을 줍니다. |
| `roe_val` | ROE(자기자본이익률) | **상** | 기업이 자기자본을 얼마나 효율적으로 사용하여 이익을 냈는지를 보여주는 대표적인 수익성 지표입니다. 높을수록 주가에 긍정적입니다. |
| `eps` | EPS(주당순이익) | **필수** | 주식 1주당 얼마의 순이익을 벌었는지를 나타내는 지표로, PER 계산의 기반이 되며 기업 가치평가의 가장 핵심적인 요소입니다. |
| `sps` | SPS(주당매출액) | **중** | 주식 1주당 얼마의 매출을 올렸는지를 나타냅니다. PSR 계산의 기반이 되며, 특히 성장주나 적자 기업의 가치 평가에 활용됩니다. |
| `bps` | BPS(주당순자산) | **상** | 주식 1주당 얼마의 순자산을 가지고 있는지를 나타냅니다. PBR 계산의 기반이 되며, 기업의 청산가치 및 재무 안정성을 보여줍니다. |
| `rsrv_rate` | 유보율 | **중** | 기업이 벌어들인 이익을 배당하지 않고 얼마나 내부에 쌓아두었는지를 나타냅니다. 재무 안정성 및 향후 투자 여력을 보여주는 지표입니다. |
| `lblt_rate` | 부채비율 | **상** | 자기자본 대비 부채의 비율로, 기업의 재무 안정성을 측정하는 대표적인 지표입니다. 부채비율이 높으면 재무 리스크가 커져 변동성에 영향을 줍니다. |

In [9]:
final_cols = [
  'ticker',
  'stac_yymm',
  'grs',
  'bsop_prfi_inrt',
  'ntin_inrt',
  'roe_val',
  'eps',
  'sps',
  'bps',
  'rsrv_rate',
  'lblt_rate'
]

In [10]:
df[final_cols]

,ticker,stac_yymm,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps,sps,bps,rsrv_rate,lblt_rate
0,005930,202506,5.2900,-33.3600,-19.6200,6.64,1920.00,45533,58114.00,43513.3600,26.3600
1,005930,202503,10.0500,1.2000,21.7400,8.16,1186.00,46778,59027.00,44197.8800,26.9900
2,005930,202412,16.2000,398.3400,122.4500,9.03,4950.00,44293,57930.00,43743.2600,27.9300
3,005930,202409,17.7500,601.0000,192.0100,9.52,3834.00,44181,55376.00,41810.4500,27.1900
4,005930,202406,17.9700,1202.7900,403.1900,8.95,2394.00,42983,55011.00,41533.8400,26.6600
...,...,...,...,...,...,...,...,...,...,...,...
5754,002710,201903,14.1600,80.4100,41.3100,7.95,95.00,17994,4928.00,418.3600,179.0600
5755,002710,201812,5.5700,110.7000,548.7400,9.19,416.00,17582,4937.00,419.2400,171.5300
5756,002710,201809,0.9200,72.6500,209.6400,7.76,257.00,17174,4768.00,401.5400,174.7500
5757,002710,201806,-1.3200,56.7500,61.0500,5.68,124.00,16827,4660.00,390.1600,178.5000


In [11]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   grs             5759 non-null   object        
 3   bsop_prfi_inrt  5759 non-null   object        
 4   ntin_inrt       5759 non-null   object        
 5   roe_val         5759 non-null   object        
 6   eps             5759 non-null   object        
 7   sps             5753 non-null   object        
 8   bps             5759 non-null   object        
 9   rsrv_rate       5759 non-null   object        
 10  lblt_rate       5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(9)
memory usage: 471.0+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `category` | `category` | 반복적으로 나타나는 종목 코드는 메모리 효율을 위해 `category` 타입으로 관리하는 것이 가장 좋습니다. |
| `stac_yymm` | 결산년월 | `datetime64[ns]` | `datetime64[ns]` | `YYYYMM` 형식의 날짜 데이터는 `pd.to_datetime` 사용 시 `format='%Y%m'`을 지정하여 `datetime`으로 변환해야 합니다. |
| `grs` | 매출액증가율 | `object` | `float64` | 소수점을 포함하고 음수 값을 가질 수 있는 '비율' 데이터이므로, 부동소수점 타입인 `float64`로 변환해야 합니다. |
| `bsop_prfi_inrt`| 영업이익증가율 | `object` | `float64` | 다른 성장률 지표와 마찬가지로 소수점 및 음수 표현이 필요한 비율 데이터이므로, `float64` 타입이 적합합니다. |
| `ntin_inrt` | 순이익증가율 | `object` | `float64` | 모든 성장률 지표의 데이터 타입을 일관성 있게 `float64`로 통일하여 데이터 처리의 안정성과 정확성을 확보합니다. |
| `roe_val` | ROE(자기자본이익률) | `object` | `float64` | 소수점을 포함하는 수익률 지표이므로 `float64` 타입으로 변환하여 관리해야 합니다. |
| `eps` | EPS(주당순이익) | `object` | `Int64` | 주당순이익은 보통 정수로 표현되므로, 결측치를 안전하게 처리할 수 있는 Nullable Integer `Int64` 타입이 적합합니다. |
| `sps` | SPS(주당매출액) | `object` | `Int64` | 주당매출액 역시 금액이므로 `Int64` 타입으로 변환하여 관리합니다. |
| `bps` | BPS(주당순자산) | `object` | `Int64` | 주당순자산은 정수 또는 소수일 수 있으나, 제공된 데이터에서는 정수형이므로 `Int64`가 효율적입니다. (소수점 존재 시 `float64`) |
| `rsrv_rate` | 유보율 | `object` | `float64` | 소수점을 포함하는 비율 데이터이므로 `float64` 타입으로 변환해야 합니다. |
| `lblt_rate` | 부채비율 | `object` | `float64` | 소수점을 포함하는 비율 데이터이므로 `float64` 타입으로 변환해야 합니다. |

In [13]:
dtype_map = {
  'Int64': ["eps", "sps", "bps"],
  'float64': ['grs', 'bsop_prfi_inrt', 'ntin_inrt', 'roe_val', "rsrv_rate", "lblt_rate"]
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   grs             5759 non-null   float64       
 3   bsop_prfi_inrt  5759 non-null   float64       
 4   ntin_inrt       5759 non-null   float64       
 5   roe_val         5759 non-null   float64       
 6   eps             5759 non-null   Int64         
 7   sps             5753 non-null   Int64         
 8   bps             5759 non-null   Int64         
 9   rsrv_rate       5759 non-null   float64       
 10  lblt_rate       5759 non-null   float64       
dtypes: Int64(3), category(1), datetime64[ns](1), float64(6)
memory usage: 487.8 KB
